In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
##upload the original mRNA and methylation
#interated mRNA and methylation (n=1417) without any operation and exclusion of features
methy_sub = pd.read_csv('/Users/mac1/Desktop/DATAFEST/methylation_sub.csv') #(1417, 13188)
mRNA_sub = pd.read_csv('/Users/mac1/Desktop/DATAFEST/mRNA_sub.csv') #(1417, 20604)
#overall mRNA (n=1980) without any operation and exclusion of features
mRNA_genetic = pd.read_csv('/Users/mac1/Desktop/DATAFEST/mRNA_genetic.csv') #(1980, 20604)

In [4]:
##extract ID row index for cohort
keep_ids_overall = mRNA_genetic['PATIENT_ID'] #overall
keep_ids_integrate = methy_sub['PATIENT_ID'] #integrate
##extract feature sets for methylation or mRNA
methy_features = [c for c in methy_sub.columns if c not in ["PATIENT_ID"]]
mRNA_features = [c for c in mRNA_sub.columns if c not in ["PATIENT_ID"]]

In [5]:
##clinical features （overall cohort)
clinical_impute_overall = pd.read_csv("/Users/mac1/Desktop/DATAFEST/clinical_imputed_1.csv")
clinical_impute_overall.shape

(1980, 29)

In [6]:
##clinical feature (integrated cohort)
clinical_impute_integrate = clinical_impute_overall[clinical_impute_overall['PATIENT_ID'].isin(keep_ids_integrate)].copy()
clinical_impute_integrate.shape

(1417, 29)

In [8]:
##gwas feature (overall cohort)
gwas=pd.read_csv('/Users/mac1/Desktop/DATAFEST/genomics_selected_gwas_ensembl.csv',index_col=0)
gwas_overall = gwas[gwas['PATIENT_ID'].isin(keep_ids_overall)].copy()

In [33]:
gwas_overall

,PATIENT_ID,ACVRL1_methy,AHNAK_methy,AKT1_methy,ALK_methy,APC_methy,ARID1A_methy,ARID1B_methy,ARID2_methy,ASXL1_methy,...,EMID1_mrna,TTC28_mrna,ATAD5_mrna,PAX9_mrna,MLLT10_mrna,PTHLH_mrna,SKIDA1_mrna,MRPS30_mrna,APOBEC3A_mrna,SEC14L4_mrna
1,MB-0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.825269,5.798647,5.762455,5.469058,7.263043,6.013555,5.608817,8.535214,5.336678,5.868042
2,MB-0002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.698300,5.609069,5.807519,6.411290,8.121593,5.764321,6.050104,9.628202,5.377759,5.747465
3,MB-0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.713216,5.485918,5.704256,5.817795,6.877284,9.808890,5.321103,8.915721,5.338215,5.554798
4,MB-0006,0.228235,0.039574,0.012346,0.032258,0.190789,0.000000,0.000000,0.000000,0.0,...,5.501439,5.356186,5.670296,6.377648,7.235799,6.089555,5.639904,9.058717,5.515105,5.881517
5,MB-0008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.018440,5.738390,5.705560,5.596711,7.947678,6.121529,5.328361,9.860345,5.375316,5.278561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,MB-7295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.598353,5.185025,5.718490,6.096267,7.250477,6.841198,5.444054,9.119662,5.289611,5.461226
1982,MB-7296,0.491018,0.065934,NaN,0.000000,0.030303,0.007895,0.007519,0.003571,0.0,...,6.195056,5.512736,5.968755,5.843940,7.143147,5.865243,5.447092,9.195065,5.441914,5.688695
1983,MB-7297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.408007,5.631185,6.096751,5.650109,7.113933,5.667898,5.381733,10.738350,5.311344,5.461644
1984,MB-7298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.629645,5.543150,5.641894,5.715491,7.449389,6.630646,5.391292,12.146450,5.371765,5.449533


In [9]:
#99 methylation in the combined overall gwas feature set, may not be used
gwas_overall_methy = gwas_overall[["PATIENT_ID"] + list(gwas_overall.columns.intersection(methy_features))]
gwas_overall_methy.shape

(1980, 99)

In [10]:
#125 mRNA in the combined overall gwas feature set
gwas_overall_mRNA = gwas_overall[["PATIENT_ID"] + list(gwas_overall.columns.intersection(mRNA_features))]
gwas_overall_mRNA.shape

(1980, 125)

In [11]:
##gwas feature (integrated cohort)
gwas_integrate = gwas[gwas['PATIENT_ID'].isin(keep_ids_integrate)].copy()
gwas_integrate

,PATIENT_ID,ACVRL1_methy,AHNAK_methy,AKT1_methy,ALK_methy,APC_methy,ARID1A_methy,ARID1B_methy,ARID2_methy,ASXL1_methy,...,EMID1_mrna,TTC28_mrna,ATAD5_mrna,PAX9_mrna,MLLT10_mrna,PTHLH_mrna,SKIDA1_mrna,MRPS30_mrna,APOBEC3A_mrna,SEC14L4_mrna
4,MB-0006,0.228235,0.039574,0.012346,0.032258,0.190789,0.000000,0.000000,0.000000,0.000000,...,5.501439,5.356186,5.670296,6.377648,7.235799,6.089555,5.639904,9.058717,5.515105,5.881517
11,MB-0028,0.194093,0.032573,0.042254,0.011905,0.110429,0.008048,0.002353,0.009881,0.000000,...,5.562532,5.819977,5.964095,5.661834,7.856708,5.466805,5.305149,8.713788,5.160860,5.739570
12,MB-0035,0.561047,0.000000,0.005587,0.000000,0.068966,0.000000,0.001468,0.003091,0.000000,...,5.496101,5.344607,5.755204,6.733679,7.641674,5.324897,5.378147,9.020583,5.407859,5.373921
16,MB-0046,0.314721,0.000000,0.000000,0.000000,0.000000,0.000000,0.001080,0.001684,0.012195,...,5.288245,5.454914,6.017270,5.939822,7.679091,6.873848,5.269159,11.045562,5.125100,5.904158
18,MB-0050,0.146341,0.049900,NaN,0.000000,0.007143,0.001410,0.000000,0.005906,0.000000,...,5.388383,5.980190,5.563753,5.534932,7.628535,7.864273,5.372053,8.434989,5.295128,5.693610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,MB-7289,0.040100,0.062780,0.025000,0.000000,0.021858,0.001196,0.003783,0.000000,0.000000,...,5.789173,5.121171,5.529888,5.608332,7.383893,10.290670,5.448643,8.446717,5.346037,5.567413
1977,MB-7291,0.114754,0.090261,0.000000,0.105882,0.127820,0.001185,0.000000,0.008565,0.000000,...,5.534877,5.431327,5.747914,5.631050,7.369142,6.034446,5.381699,8.832027,5.445283,5.470318
1978,MB-7292,0.360996,0.037288,NaN,0.028571,0.222222,0.006309,0.007782,0.028571,0.000000,...,5.346964,5.484218,5.924776,5.682275,7.246598,6.546055,5.612173,8.947196,5.369745,5.362175
1979,MB-7293,0.269841,0.017578,0.017857,0.252336,0.012579,0.001594,0.000000,0.000000,0.000000,...,5.794616,5.494233,5.752061,6.126150,7.231557,6.389416,5.632143,10.417480,5.343353,5.425920


In [12]:
gwas_integrate_methy = gwas_integrate[["PATIENT_ID"] + list(gwas_integrate.columns.intersection(methy_features))]
gwas_integrate_methy

,PATIENT_ID,ACVRL1_methy,AHNAK_methy,AKT1_methy,ALK_methy,APC_methy,ARID1A_methy,ARID1B_methy,ARID2_methy,ASXL1_methy,...,TCN2_methy,TEFM_methy,TERT_methy,TNNT3_methy,TOX3_methy,TRAK2_methy,TTC28_methy,ZMIZ1_methy,ZNF365_methy,ZNRF3_methy
4,MB-0006,0.228235,0.039574,0.012346,0.032258,0.190789,0.000000,0.000000,0.000000,0.000000,...,0.068493,0.000000,0.258065,0.641791,0.000000,0.000000,0.002174,0.000000,0.040219,0.000000
11,MB-0028,0.194093,0.032573,0.042254,0.011905,0.110429,0.008048,0.002353,0.009881,0.000000,...,0.000000,0.007712,0.320847,0.649123,0.016484,0.003891,0.008830,0.008696,0.006780,0.000000
12,MB-0035,0.561047,0.000000,0.005587,0.000000,0.068966,0.000000,0.001468,0.003091,0.000000,...,0.022989,0.000000,0.676724,0.727273,0.066339,0.002049,0.002849,0.000000,0.121951,0.000000
16,MB-0046,0.314721,0.000000,0.000000,0.000000,0.000000,0.000000,0.001080,0.001684,0.012195,...,0.023256,0.003044,0.150763,0.612245,0.002747,0.000000,0.000000,0.000000,0.003333,0.000000
18,MB-0050,0.146341,0.049900,NaN,0.000000,0.007143,0.001410,0.000000,0.005906,0.000000,...,0.000000,0.002611,0.184486,0.790698,0.000000,0.001996,0.001276,0.000000,0.022556,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,MB-7289,0.040100,0.062780,0.025000,0.000000,0.021858,0.001196,0.003783,0.000000,0.000000,...,0.000000,0.000000,0.028926,0.519481,0.012526,0.003856,0.000000,0.000000,0.047619,0.000000
1977,MB-7291,0.114754,0.090261,0.000000,0.105882,0.127820,0.001185,0.000000,0.008565,0.000000,...,0.125000,0.002119,0.103604,0.529412,0.059353,0.000000,0.003289,0.000000,0.066327,0.010836
1978,MB-7292,0.360996,0.037288,NaN,0.028571,0.222222,0.006309,0.007782,0.028571,0.000000,...,0.000000,0.017391,0.668919,NaN,0.029412,0.005556,0.015873,0.000000,0.006849,0.000000
1979,MB-7293,0.269841,0.017578,0.017857,0.252336,0.012579,0.001594,0.000000,0.000000,0.000000,...,0.024390,0.006757,0.121911,0.813953,0.000000,0.000000,0.004193,0.004292,0.068293,0.003390


In [13]:
gwas_integrate_mRNA = gwas_integrate[["PATIENT_ID"] + list(gwas_integrate.columns.intersection(mRNA_features))]
gwas_integrate_mRNA

,PATIENT_ID,AGMO_mrna,AHNAK_mrna,ALK_mrna,LAMB3_mrna,SMAD4_mrna,CDKN1B_mrna,EGFR_mrna,NF2_mrna,SETD2_mrna,...,EMID1_mrna,TTC28_mrna,ATAD5_mrna,PAX9_mrna,MLLT10_mrna,PTHLH_mrna,SKIDA1_mrna,MRPS30_mrna,APOBEC3A_mrna,SEC14L4_mrna
4,MB-0006,5.462942,11.511308,5.489501,5.898610,8.757572,10.380481,5.585357,5.408832,7.605424,...,5.501439,5.356186,5.670296,6.377648,7.235799,6.089555,5.639904,9.058717,5.515105,5.881517
11,MB-0028,5.343175,11.064371,5.597277,5.433681,8.496236,8.307517,5.502281,5.366654,9.309069,...,5.562532,5.819977,5.964095,5.661834,7.856708,5.466805,5.305149,8.713788,5.160860,5.739570
12,MB-0035,5.340526,11.787179,5.832625,6.112071,7.907685,7.890494,5.466188,5.357428,8.972442,...,5.496101,5.344607,5.755204,6.733679,7.641674,5.324897,5.378147,9.020583,5.407859,5.373921
16,MB-0046,5.358802,11.311807,5.567264,7.060030,8.168369,7.136789,5.825170,5.479556,9.169579,...,5.288245,5.454914,6.017270,5.939822,7.679091,6.873848,5.269159,11.045562,5.125100,5.904158
18,MB-0050,5.283746,12.004970,5.696334,7.409536,8.799622,7.416582,7.094461,5.544637,8.758379,...,5.388383,5.980190,5.563753,5.534932,7.628535,7.864273,5.372053,8.434989,5.295128,5.693610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,MB-7289,5.429778,11.567410,5.479520,8.470224,8.662044,7.592850,6.666688,5.586575,8.824639,...,5.789173,5.121171,5.529888,5.608332,7.383893,10.290670,5.448643,8.446717,5.346037,5.567413
1977,MB-7291,5.333743,10.877760,5.498422,6.400490,8.295540,7.528250,6.437883,5.668293,8.348152,...,5.534877,5.431327,5.747914,5.631050,7.369142,6.034446,5.381699,8.832027,5.445283,5.470318
1978,MB-7292,5.528522,10.810070,5.760783,5.687461,8.638073,7.473152,5.917815,5.604277,8.421878,...,5.346964,5.484218,5.924776,5.682275,7.246598,6.546055,5.612173,8.947196,5.369745,5.362175
1979,MB-7293,5.341804,11.904170,5.739238,6.408341,8.785697,8.211591,5.825643,5.737924,8.223599,...,5.794616,5.494233,5.752061,6.126150,7.231557,6.389416,5.632143,10.417480,5.343353,5.425920


In [14]:
nan_counts_mRNA = gwas_overall_mRNA.isna().sum()
nan_counts_meth = gwas_integrate_methy.isna().sum()
missing_pct_mRNA= gwas_overall_mRNA.isna().mean() * 100
missing_pct_meth= gwas_integrate_methy.isna().mean() * 100
print("Columns with NaN in mRNA in GWAS overall:", (nan_counts_mRNA > 0).sum())
print("Columns with NaN in methylation in GWAS integrated:", (nan_counts_meth > 0).sum())
print("Percentage of missing in mRNA in GWAS overall",missing_pct_mRNA.describe())
print("Percentage of missing in methylation in GWAS integrated",missing_pct_meth.describe())
##the missingness percentage is acceptable, let's move on for KNN imputation

Columns with NaN in mRNA in GWAS overall: 0
Columns with NaN in methylation in GWAS integrated: 75
Percentage of missing in mRNA in GWAS overall count    125.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
Percentage of missing in methylation in GWAS integrated count    99.000000
mean      3.758830
std       6.782153
min       0.000000
25%       0.070572
50%       0.423430
75%       3.705011
max      28.863797
dtype: float64


In [15]:
##data-driven feature set
#integrated cohort case 1 (n=1417, N=200)
var_data_driven_integrate_1 = pd.read_csv('/Users/mac1/Desktop/DATAFEST/strategy_3_Importance_Cohort1_Case1_SeparateCox.csv')
top200_features_1 = var_data_driven_integrate_1["feature_name"].head(200).tolist()
##integrated cohort case 2 (n=1417, N=300)
var_data_driven_integrate_2 = pd.read_csv('/Users/mac1/Desktop/DATAFEST/strategy_3_Importance_Cohort1_Case2_CombinedCox.csv')
top300_features_2 = var_data_driven_integrate_2["feature_name"].head(300).tolist()
#overall cohort (n=1980, N mRNA=100)
var_data_driven_overall = pd.read_csv('/Users/mac1/Desktop/DATAFEST/strategy_3_Importance_C2_Case1_mRNA_only.csv')
top100_features_3 = var_data_driven_overall["feature_name"].head(100).tolist()

In [16]:
##data-driven dataset (integrate cohort case 1)
genetic_sub = methy_sub.merge(mRNA_sub, on="PATIENT_ID", how="inner")
data_driven_integrate_1 = genetic_sub[["PATIENT_ID"] + list(genetic_sub.columns.intersection(top200_features_1))]
##data-driven dataset (integrate cohort case 2)
data_driven_integrate_2 = genetic_sub[["PATIENT_ID"] + list(genetic_sub.columns.intersection(top300_features_2))]
##data-driven dataset (overall cohort)
data_driven_overall = mRNA_genetic[["PATIENT_ID"] + list(mRNA_genetic.columns.intersection(top100_features_3))]

In [22]:
cols = ["PATIENT_ID"] + [c for c in data_driven_integrate_1.columns if c.endswith("_mrna")]
data_driven_integrate_1_mrna = data_driven_integrate_1[cols]
data_driven_integrate_1_mrna

,PATIENT_ID,LINC00937_mrna,TAT_mrna,C14orf184_mrna,GLA_mrna,SLC16A3_mrna,IGFN1_mrna,DUSP15_mrna,DNASE1L3_mrna,INPP5A_mrna,...,ENC1_mrna,SH3BGRL2_mrna,SLC38A1_mrna,HILPDA_mrna,ALDOC_mrna,GRB2_mrna,MMP3_mrna,VANGL1_mrna,HMGB3_mrna,EXPH5_mrna
0,MB-0006,5.652845,5.746114,5.757450,8.485949,9.983743,5.706636,5.719950,5.644084,7.895471,...,7.690706,7.451934,9.537609,7.726851,7.130907,6.655574,6.375535,5.368648,7.702250,5.852908
1,MB-0028,5.408407,5.433514,5.689489,8.731710,8.608753,5.613722,5.440101,5.163537,7.682070,...,7.401007,6.445898,10.360132,7.323750,7.429350,10.484214,5.335386,5.495902,8.134939,6.638296
2,MB-0035,5.525525,5.563156,6.037567,8.149366,6.730225,5.874200,5.353390,5.365097,8.872877,...,8.611371,6.614811,11.006116,7.551945,6.484451,6.771495,5.717184,5.609675,7.148153,7.305261
3,MB-0046,5.520658,5.556175,6.511966,9.690913,8.949934,5.417182,5.469190,5.181842,7.397672,...,8.526717,7.703336,7.738962,7.294387,8.326490,9.456882,6.112901,6.165212,8.016967,5.850510
4,MB-0050,5.553870,7.874150,6.072143,9.028392,7.074297,5.399167,5.223366,6.589710,7.625433,...,8.284187,8.118200,10.092764,6.739002,8.355693,7.460314,7.397655,5.381572,7.639669,7.079205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,MB-7289,5.311013,6.588677,5.627716,9.365496,7.828994,5.240209,5.473240,6.396629,7.374513,...,8.609249,7.780088,10.208355,6.374919,8.955945,7.438374,8.345558,5.551011,6.389354,6.075920
1413,MB-7291,5.639454,5.806753,5.801780,9.135438,9.178206,5.490531,5.689773,7.174227,7.855154,...,8.796033,7.243271,9.858089,6.272637,7.444408,8.620689,7.491018,6.134674,8.126522,6.458758
1414,MB-7292,5.320487,5.475669,5.907214,8.669601,10.047511,5.196646,5.485982,5.566037,7.430706,...,8.587751,6.867430,10.248435,7.438962,7.759453,7.811448,6.195857,5.710864,8.460674,6.164186
1415,MB-7293,5.472402,5.459333,5.476251,9.409805,8.348674,5.071341,5.424013,6.719742,7.437116,...,8.656620,7.449284,9.302515,6.397087,7.558685,7.019974,8.311320,5.593818,6.092144,6.310746


In [23]:
cols = ["PATIENT_ID"] + [c for c in data_driven_integrate_2.columns if c.endswith("_mrna")]
data_driven_integrate_2_mrna = data_driven_integrate_2[cols]
data_driven_integrate_2_mrna

,PATIENT_ID,SHMT2_mrna,KRT8_mrna,RANBP6_mrna,C14orf184_mrna,GLA_mrna,ZNF214_mrna,EFR3A_mrna,ANKAR_mrna,KIAA1958_mrna,...,DNAH11_mrna,ALDOC_mrna,GRB2_mrna,MMP3_mrna,SELENBP1_mrna,GPR156_mrna,VANGL1_mrna,NKX2-2_mrna,NCEH1_mrna,UMODL1_mrna
0,MB-0006,9.431167,12.925422,6.761162,5.757450,8.485949,5.956145,10.742071,5.758045,5.509959,...,5.157518,7.130907,6.655574,6.375535,7.660406,5.804127,5.368648,5.429745,6.994440,5.443783
1,MB-0028,10.367760,10.294378,6.343706,5.689489,8.731710,5.486175,10.219419,5.832128,6.035359,...,5.210486,7.429350,10.484214,5.335386,8.973843,5.705585,5.495902,5.330070,6.666313,5.353278
2,MB-0035,9.837096,9.838600,6.414310,6.037567,8.149366,5.793395,9.525547,5.827017,6.157515,...,5.232120,6.484451,6.771495,5.717184,10.820817,5.658895,5.609675,6.226791,8.741210,7.520124
3,MB-0046,10.387976,11.794499,6.072853,6.511966,9.690913,5.655006,9.368818,5.755084,5.769001,...,5.282381,8.326490,9.456882,6.112901,9.732011,5.686241,6.165212,5.496586,7.395477,5.470029
4,MB-0050,9.488406,9.456052,6.514110,6.072143,9.028392,5.567433,8.983147,5.911851,5.652870,...,5.327767,8.355693,7.460314,7.397655,9.311124,5.677137,5.381572,5.323528,6.517581,5.254414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,MB-7289,9.802586,10.208570,6.732579,5.627716,9.365496,5.435067,8.996555,5.777710,5.708150,...,5.295160,8.955945,7.438374,8.345558,8.963071,5.424336,5.551011,5.462697,6.555181,5.689578
1413,MB-7291,10.365286,10.840858,6.050577,5.801780,9.135438,5.451489,9.861437,5.711542,5.462414,...,5.297712,7.444408,8.620689,7.491018,6.819179,5.388680,6.134674,5.553103,7.038777,5.676197
1414,MB-7292,10.063176,10.210223,6.100147,5.907214,8.669601,5.431562,9.570582,5.638522,5.861783,...,5.296351,7.759453,7.811448,6.195857,8.892469,5.527376,5.710864,5.343891,6.952544,5.645671
1415,MB-7293,9.271953,9.792816,6.604081,5.476251,9.409805,5.853228,9.328939,5.909755,5.524639,...,5.341369,7.558685,7.019974,8.311320,8.512068,5.601245,5.593818,5.439948,6.963754,5.300404


In [24]:
ID_COL = "PATIENT_ID"

def drop_overlap_keep_gwas(gwas_df, dd_df, id_col=ID_COL):
    overlap = (set(gwas_df.columns) & set(dd_df.columns)) - {id_col}
    if overlap:
        print(f"Dropping {len(overlap)} overlapping columns from data-driven set (keeping GWAS).")
        dd_df = dd_df.drop(columns=list(overlap))
    return dd_df

# integrate cohort case 1
data_driven_integrate_1 = drop_overlap_keep_gwas(gwas_integrate, data_driven_integrate_1)

# integrate cohort case 2
data_driven_integrate_2 = drop_overlap_keep_gwas(gwas_integrate, data_driven_integrate_2)

# overall cohort (match the GWAS df you merge with)
data_driven_overall = drop_overlap_keep_gwas(gwas_overall_mRNA, data_driven_overall)

Dropping 3 overlapping columns from data-driven set (keeping GWAS).
Dropping 4 overlapping columns from data-driven set (keeping GWAS).
Dropping 1 overlapping columns from data-driven set (keeping GWAS).


In [25]:
##Combine clinical+gwas+data-driven
#integrate cohort gwas （n=1417，N=252)
data_all_integrate_gwas = clinical_impute_integrate.merge(gwas_integrate, on="PATIENT_ID", how="inner") #clinical+gwas
#integrate cohort case 1 (N=1417,N=449)
data_all_integrate_1 = clinical_impute_integrate.merge(gwas_integrate, on="PATIENT_ID", how="inner") #clinical+gwas
data_all_integrate_1 = data_all_integrate_1.merge(data_driven_integrate_1, on="PATIENT_ID", how="inner") # +data-driven
data_all_integrate_1cli = clinical_impute_integrate.merge(data_driven_integrate_1, on="PATIENT_ID", how="inner")  #clinical+gwas
#integrate cohort case 2 (n=1417, N=548)
data_all_integrate_2 = clinical_impute_integrate.merge(gwas_integrate, on="PATIENT_ID", how="inner") #clinical+gwas
data_all_integrate_2 = data_all_integrate_2.merge(data_driven_integrate_2, on="PATIENT_ID", how="inner") # +data-driven
data_all_integrate_2cli = clinical_impute_integrate.merge(data_driven_integrate_2, on="PATIENT_ID", how="inner")
#overall cohort (n=1980, N=252)
data_all_overall_mrna = clinical_impute_overall.merge(gwas_overall_mRNA, on="PATIENT_ID", how="inner") #clinical+gwas (mrna)
data_all_overall = data_all_overall_mrna.merge(data_driven_overall, on="PATIENT_ID", how="inner") # +data-driven
#integrate cohort gwas （n=1980，N=153)
data_all_overall_gwas = clinical_impute_overall.merge(gwas_overall_mRNA, on="PATIENT_ID", how="inner") #clinical+gwas
#integrate cohort case1 (mrna only)
data_driven_integrate_1_mrna = clinical_impute_integrate.merge(data_driven_integrate_1_mrna, on="PATIENT_ID", how="inner")
data_driven_integrate_2_mrna = clinical_impute_integrate.merge(data_driven_integrate_2_mrna, on="PATIENT_ID", how="inner")

In [28]:
##split into training and testing (70% vs 30%)
#integrate cohort gwas （n=1417，N=252)
train_all_integrate_gwas, test_all_integrate_gwas = train_test_split(
    data_all_integrate_gwas,
    test_size=0.15,
    stratify=data_all_integrate_gwas['OS_STATUS'],
    random_state=123
)
X_train_integrate_gwas = train_all_integrate_gwas.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_integrate_gwas = train_all_integrate_gwas[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]
X_test_integrate_gwas = test_all_integrate_gwas.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_integrate_gwas = test_all_integrate_gwas[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]

#integrate cohort case 1 (N=1417,N=449)
train_all_integrate_1, test_all_integrate_1 = train_test_split(
    data_all_integrate_1,
    test_size=0.15,
    stratify=data_all_integrate_1['OS_STATUS'],
    random_state=123
)
X_train_integrate_1 = train_all_integrate_1.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_integrate_1 = train_all_integrate_1[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]
X_test_integrate_1 = test_all_integrate_1.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_integrate_1 = test_all_integrate_1[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]

#integrate cohort case 1 +clinical (n=1417,N=226)
train_all_integrate_1cli, test_all_integrate_1cli = train_test_split(
    data_all_integrate_1cli,
    test_size=0.15,
    stratify=data_all_integrate_1cli['OS_STATUS'],
    random_state=123
)
X_train_integrate_1cli = train_all_integrate_1cli.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_integrate_1cli = train_all_integrate_1cli[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]
X_test_integrate_1cli = test_all_integrate_1cli.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_integrate_1cli = test_all_integrate_1cli[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]


#integrate cohort case 2 (N=1417,N=548)
train_all_integrate_2, test_all_integrate_2 = train_test_split(
    data_all_integrate_2,
    test_size=0.15,
    stratify=data_all_integrate_2['OS_STATUS'],
    random_state=123
)
X_train_integrate_2 = train_all_integrate_2.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_integrate_2 = train_all_integrate_2[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]
X_test_integrate_2 = test_all_integrate_2.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_integrate_2 = test_all_integrate_2[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]

#integrate cohort case 2 +clinical (n=1417,N=326)
train_all_integrate_2cli, test_all_integrate_2cli = train_test_split(
    data_all_integrate_2cli,
    test_size=0.15,
    stratify=data_all_integrate_2cli['OS_STATUS'],
    random_state=123
)
X_train_integrate_2cli = train_all_integrate_2cli.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_integrate_2cli = train_all_integrate_2cli[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]
X_test_integrate_2cli = test_all_integrate_2cli.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_integrate_2cli = test_all_integrate_2cli[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]

#overall cohort mRNA+GWAS (N=1980,N=251)
train_all_overall, test_all_overall = train_test_split(
    data_all_overall,
    test_size=0.15,
    stratify=data_all_overall['OS_STATUS'],
    random_state=123
)
X_train_overall = train_all_overall.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_overall = train_all_overall[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]
X_test_overall = test_all_overall.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_overall = test_all_overall[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]

#overall cohort clinical+mRNA (N=1980,N=251)
train_all_overall_mrna, test_all_overall_mrna = train_test_split(
    data_all_overall_mrna,
    test_size=0.15,
    stratify=data_all_overall_mrna['OS_STATUS'],
    random_state=123
)
X_train_overall_mrna = train_all_overall_mrna.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_overall_mrna = train_all_overall_mrna[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]
X_test_overall_mrna = test_all_overall_mrna.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_overall_mrna = test_all_overall_mrna[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]

#integrate cohort gwas （n=1980，N=153)
train_all_overall_gwas, test_all_overall_gwas = train_test_split(
    data_all_overall_gwas,
    test_size=0.15,
    stratify=data_all_overall_gwas['OS_STATUS'],
    random_state=123
)
X_train_overall_gwas = train_all_overall_gwas.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_overall_gwas = train_all_overall_gwas[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]
X_test_overall_gwas = test_all_overall_gwas.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_overall_gwas = test_all_overall_gwas[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]

#integrate cohort case1(mrna only)
train_all_integrate_1_mrna, test_all_integrate_1_mrna = train_test_split(
    data_driven_integrate_1_mrna,
    test_size=0.15,
    stratify=data_driven_integrate_1_mrna['OS_STATUS'],
    random_state=123
)
X_train_integrate_1_mrna = train_all_integrate_1_mrna.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_integrate_1_mrna = train_all_integrate_1_mrna[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]
X_test_integrate_1_mrna = test_all_integrate_1_mrna.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_integrate_1_mrna = test_all_integrate_1_mrna[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]

#integrate cohort case2(mrna only)
train_all_integrate_2_mrna, test_all_integrate_2_mrna = train_test_split(
    data_driven_integrate_2_mrna,
    test_size=0.15,
    stratify=data_driven_integrate_2_mrna['OS_STATUS'],
    random_state=123
)
X_train_integrate_2_mrna = train_all_integrate_2_mrna.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_train_integrate_2_mrna = train_all_integrate_2_mrna[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]
X_test_integrate_2_mrna = test_all_integrate_2_mrna.drop(columns=['PATIENT_ID', 'OS_MONTHS', 'OS_STATUS'])
y_test_integrate_2_mrna = test_all_integrate_2_mrna[['PATIENT_ID','OS_MONTHS', 'OS_STATUS']]



In [43]:
## KNN imputation (genetic only + std first)
#integrate cohort gwas (N=1417,N=252)
Xc_train_integrate_gwas = X_train_integrate_gwas.iloc[:, :26]
Xg_train_integrate_gwas = X_train_integrate_gwas.iloc[:, 26:] #only impute on training
Xc_test_integrate_gwas  = X_test_integrate_gwas.iloc[:, :26]
Xg_test_integrate_gwas  = X_test_integrate_gwas.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_integrate_gwas_scaled = scaler.fit_transform(Xg_train_integrate_gwas) #std first

imputer = KNNImputer(n_neighbors=5,weights="distance",metric="nan_euclidean") #knn
Xg_train_integrate_gwas_imp = imputer.fit_transform(Xg_train_integrate_gwas_scaled)

Xg_train_integrate_gwas_imp = pd.DataFrame( #get columns' name back
    Xg_train_integrate_gwas_imp,
    columns=Xg_train_integrate_gwas.columns,
    index=Xg_train_integrate_gwas.index)

X_train_integrate_gwas_imp = pd.concat([Xc_train_integrate_gwas, Xg_train_integrate_gwas_imp], axis=1)

Xg_test_integrate_gwas_scaled = scaler.transform(Xg_test_integrate_gwas) #Scale and std test using TRAIN statistics
Xg_test_integrate_gwas_imp = imputer.transform(Xg_test_integrate_gwas_scaled)
Xg_test_integrate_gwas_imp = pd.DataFrame(
    Xg_test_integrate_gwas_imp,
    columns=Xg_test_integrate_gwas.columns,
    index=Xg_test_integrate_gwas.index
)
X_test_integrate_gwas_imp = pd.concat([Xc_test_integrate_gwas, Xg_test_integrate_gwas_imp], axis=1)

##add ID back
X_train_integrate_gwas_imp["PATIENT_ID"] = train_all_integrate_gwas["PATIENT_ID"].values
X_test_integrate_gwas_imp["PATIENT_ID"]  = test_all_integrate_gwas["PATIENT_ID"].values
def move_id_first(df, id_col="PATIENT_ID"): #move ID to the first column
    cols = [id_col] + [c for c in df.columns if c != id_col]
    return df[cols]
X_train_integrate_gwas_imp = move_id_first(X_train_integrate_gwas_imp)
X_test_integrate_gwas_imp  = move_id_first(X_test_integrate_gwas_imp)

In [28]:
## KNN imputation (genetic only + std first)
#integrate cohort case 1 (N=1417,N=446)
Xc_train_integrate_1 = X_train_integrate_1.iloc[:, :26]
Xg_train_integrate_1 = X_train_integrate_1.iloc[:, 26:] #only impute on training
Xc_test_integrate_1  = X_test_integrate_1.iloc[:, :26]
Xg_test_integrate_1  = X_test_integrate_1.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_integrate_1_scaled = scaler.fit_transform(Xg_train_integrate_1) #std first

imputer = KNNImputer(n_neighbors=5,weights="distance",metric="nan_euclidean") #knn
Xg_train_integrate_1_imp = imputer.fit_transform(Xg_train_integrate_1_scaled)

Xg_train_integrate_1_imp = pd.DataFrame( #get columns' name back
    Xg_train_integrate_1_imp,
    columns=Xg_train_integrate_1.columns,
    index=Xg_train_integrate_1.index)

X_train_integrate_1_imp = pd.concat([Xc_train_integrate_1, Xg_train_integrate_1_imp], axis=1)

Xg_test_integrate_1_scaled = scaler.transform(Xg_test_integrate_1) #Scale and std test using TRAIN statistics
Xg_test_integrate_1_imp = imputer.transform(Xg_test_integrate_1_scaled)
Xg_test_integrate_1_imp = pd.DataFrame(
    Xg_test_integrate_1_imp,
    columns=Xg_test_integrate_1.columns,
    index=Xg_test_integrate_1.index
)
X_test_integrate_1_imp = pd.concat([Xc_test_integrate_1, Xg_test_integrate_1_imp], axis=1)

##add ID back
X_train_integrate_1_imp["PATIENT_ID"] = train_all_integrate_1["PATIENT_ID"].values
X_test_integrate_1_imp["PATIENT_ID"]  = test_all_integrate_1["PATIENT_ID"].values
def move_id_first(df, id_col="PATIENT_ID"): #move ID to the first column
    cols = [id_col] + [c for c in df.columns if c != id_col]
    return df[cols]
X_train_integrate_1_imp = move_id_first(X_train_integrate_1_imp)
X_test_integrate_1_imp  = move_id_first(X_test_integrate_1_imp)

In [60]:
## KNN imputation (genetic only + std first)
#integrate cohort case 1 + clinical (N=1417,N=446)
Xc_train_integrate_1cli = X_train_integrate_1cli.iloc[:, :26]
Xg_train_integrate_1cli = X_train_integrate_1cli.iloc[:, 26:]  #only impute on training
Xc_test_integrate_1cli = X_test_integrate_1cli.iloc[:, :26]
Xg_test_integrate_1cli = X_test_integrate_1cli.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_integrate_1cli_scaled = scaler.fit_transform(Xg_train_integrate_1cli)  #std first

imputer = KNNImputer(n_neighbors=5, weights="distance", metric="nan_euclidean")  #knn
Xg_train_integrate_1cli_imp = imputer.fit_transform(Xg_train_integrate_1cli_scaled)

Xg_train_integrate_1cli_imp = pd.DataFrame(  #get columns' name back
    Xg_train_integrate_1cli_imp,
    columns=Xg_train_integrate_1cli.columns,
    index=Xg_train_integrate_1cli.index)

X_train_integrate_1cli_imp = pd.concat([Xc_train_integrate_1cli, Xg_train_integrate_1cli_imp], axis=1)

Xg_test_integrate_1cli_scaled = scaler.transform(Xg_test_integrate_1cli)  #Scale and std test using TRAIN statistics
Xg_test_integrate_1cli_imp = imputer.transform(Xg_test_integrate_1cli_scaled)
Xg_test_integrate_1cli_imp = pd.DataFrame(
    Xg_test_integrate_1cli_imp,
    columns=Xg_test_integrate_1cli.columns,
    index=Xg_test_integrate_1cli.index
)
X_test_integrate_1cli_imp = pd.concat([Xc_test_integrate_1cli, Xg_test_integrate_1cli_imp], axis=1)

##add ID back
X_train_integrate_1cli_imp["PATIENT_ID"] = train_all_integrate_1cli["PATIENT_ID"].values
X_test_integrate_1cli_imp["PATIENT_ID"] = test_all_integrate_1cli["PATIENT_ID"].values


def move_id_first(df, id_col="PATIENT_ID"):  #move ID to the first column
    cols = [id_col] + [c for c in df.columns if c != id_col]
    return df[cols]


X_train_integrate_1cli_imp = move_id_first(X_train_integrate_1cli_imp)
X_test_integrate_1cli_imp = move_id_first(X_test_integrate_1cli_imp)

In [29]:
## KNN imputation (genetic only + std first)
#integrate cohort case 2 (N=1417,N=545)
Xc_train_integrate_2 = X_train_integrate_2.iloc[:, :26]
Xg_train_integrate_2 = X_train_integrate_2.iloc[:, 26:]  #only impute on training
Xc_test_integrate_2 = X_test_integrate_2.iloc[:, :26]
Xg_test_integrate_2 = X_test_integrate_2.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_integrate_2_scaled = scaler.fit_transform(Xg_train_integrate_2)  #std first

imputer = KNNImputer(n_neighbors=5, weights="distance", metric="nan_euclidean")  #knn
Xg_train_integrate_2_imp = imputer.fit_transform(Xg_train_integrate_2_scaled)

Xg_train_integrate_2_imp = pd.DataFrame(  #get columns' name back
    Xg_train_integrate_2_imp,
    columns=Xg_train_integrate_2.columns,
    index=Xg_train_integrate_2.index)

X_train_integrate_2_imp = pd.concat([Xc_train_integrate_2, Xg_train_integrate_2_imp], axis=1)

Xg_test_integrate_2_scaled = scaler.transform(Xg_test_integrate_2)  #Scale and std test using TRAIN statistics
Xg_test_integrate_2_imp = imputer.transform(Xg_test_integrate_2_scaled)
Xg_test_integrate_2_imp = pd.DataFrame(
    Xg_test_integrate_2_imp,
    columns=Xg_test_integrate_2.columns,
    index=Xg_test_integrate_2.index
)
X_test_integrate_2_imp = pd.concat([Xc_test_integrate_2, Xg_test_integrate_2_imp], axis=1)

##add ID back
X_train_integrate_2_imp["PATIENT_ID"] = train_all_integrate_2["PATIENT_ID"].values
X_test_integrate_2_imp["PATIENT_ID"]  = test_all_integrate_2["PATIENT_ID"].values
X_train_integrate_2_imp = move_id_first(X_train_integrate_2_imp)
X_test_integrate_2_imp  = move_id_first(X_test_integrate_2_imp)

In [62]:
## KNN imputation (genetic only + std first)
#integrate cohort case 1 + clinical (N=1417,N=446)
Xc_train_integrate_2cli = X_train_integrate_2cli.iloc[:, :26]
Xg_train_integrate_2cli = X_train_integrate_2cli.iloc[:, 26:]  #only impute on training
Xc_test_integrate_2cli = X_test_integrate_2cli.iloc[:, :26]
Xg_test_integrate_2cli = X_test_integrate_2cli.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_integrate_2cli_scaled = scaler.fit_transform(Xg_train_integrate_2cli)  #std first

imputer = KNNImputer(n_neighbors=5, weights="distance", metric="nan_euclidean")  #knn
Xg_train_integrate_2cli_imp = imputer.fit_transform(Xg_train_integrate_2cli_scaled)

Xg_train_integrate_2cli_imp = pd.DataFrame(  #get columns' name back
    Xg_train_integrate_2cli_imp,
    columns=Xg_train_integrate_2cli.columns,
    index=Xg_train_integrate_2cli.index)

X_train_integrate_2cli_imp = pd.concat([Xc_train_integrate_2cli, Xg_train_integrate_2cli_imp], axis=1)

Xg_test_integrate_2cli_scaled = scaler.transform(Xg_test_integrate_2cli)  #Scale and std test using TRAIN statistics
Xg_test_integrate_2cli_imp = imputer.transform(Xg_test_integrate_2cli_scaled)
Xg_test_integrate_2cli_imp = pd.DataFrame(
    Xg_test_integrate_2cli_imp,
    columns=Xg_test_integrate_2cli.columns,
    index=Xg_test_integrate_2cli.index
)
X_test_integrate_2cli_imp = pd.concat([Xc_test_integrate_2cli, Xg_test_integrate_2cli_imp], axis=1)

##add ID back
X_train_integrate_2cli_imp["PATIENT_ID"] = train_all_integrate_2cli["PATIENT_ID"].values
X_test_integrate_2cli_imp["PATIENT_ID"] = test_all_integrate_2cli["PATIENT_ID"].values


def move_id_first(df, id_col="PATIENT_ID"):  #move ID to the first column
    cols = [id_col] + [c for c in df.columns if c != id_col]
    return df[cols]


X_train_integrate_2cli_imp = move_id_first(X_train_integrate_2cli_imp)
X_test_integrate_2cli_imp = move_id_first(X_test_integrate_2cli_imp)

In [30]:
## KNN imputation (genetic only + std first)
#overall cohort (N=1980,N=248)
Xc_train_overall = X_train_overall.iloc[:, :26]
Xg_train_overall = X_train_overall.iloc[:, 26:]  #only impute on training
Xc_test_overall = X_test_overall.iloc[:, :26]
Xg_test_overall = X_test_overall.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_overall_scaled = scaler.fit_transform(Xg_train_overall)  #std first

imputer = KNNImputer(n_neighbors=5, weights="distance", metric="nan_euclidean")  #knn
Xg_train_overall_imp = imputer.fit_transform(Xg_train_overall_scaled)

Xg_train_overall_imp = pd.DataFrame(  #get columns' name back
    Xg_train_overall_imp,
    columns=Xg_train_overall.columns,
    index=Xg_train_overall.index)

X_train_overall_imp = pd.concat([Xc_train_overall, Xg_train_overall_imp], axis=1)

Xg_test_overall_scaled = scaler.transform(Xg_test_overall)  #Scale and std test using TRAIN statistics
Xg_test_overall_imp = imputer.transform(Xg_test_overall_scaled)
Xg_test_overall_imp = pd.DataFrame(
    Xg_test_overall_imp,
    columns=Xg_test_overall.columns,
    index=Xg_test_overall.index
)
X_test_overall_imp = pd.concat([Xc_test_overall, Xg_test_overall_imp], axis=1)

##add ID back
X_train_overall_imp["PATIENT_ID"] = train_all_overall["PATIENT_ID"].values
X_test_overall_imp["PATIENT_ID"]  = test_all_overall["PATIENT_ID"].values
X_train_overall_imp = move_id_first(X_train_overall_imp)
X_test_overall_imp  = move_id_first(X_test_overall_imp)

In [68]:
## KNN imputation (genetic only + std first)
#overall cohort mrna (N=1980,N=248)
Xc_train_overall_mrna = X_train_overall_mrna.iloc[:, :26]
Xg_train_overall_mrna = X_train_overall_mrna.iloc[:, 26:]  #only impute on training
Xc_test_overall_mrna = X_test_overall_mrna.iloc[:, :26]
Xg_test_overall_mrna = X_test_overall_mrna.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_overall_mrna_scaled = scaler.fit_transform(Xg_train_overall_mrna)  #std first

imputer = KNNImputer(n_neighbors=5, weights="distance", metric="nan_euclidean")  #knn
Xg_train_overall_mrna_imp = imputer.fit_transform(Xg_train_overall_mrna_scaled)

Xg_train_overall_mrna_imp = pd.DataFrame(  #get columns' name back
    Xg_train_overall_mrna_imp,
    columns=Xg_train_overall_mrna.columns,
    index=Xg_train_overall_mrna.index)

X_train_overall_mrna_imp = pd.concat([Xc_train_overall_mrna, Xg_train_overall_mrna_imp], axis=1)

Xg_test_overall_mrna_scaled = scaler.transform(Xg_test_overall_mrna)  #Scale and std test using TRAIN statistics
Xg_test_overall_mrna_imp = imputer.transform(Xg_test_overall_mrna_scaled)
Xg_test_overall_mrna_imp = pd.DataFrame(
    Xg_test_overall_mrna_imp,
    columns=Xg_test_overall_mrna.columns,
    index=Xg_test_overall_mrna.index
)
X_test_overall_mrna_imp = pd.concat([Xc_test_overall_mrna, Xg_test_overall_mrna_imp], axis=1)

##add ID back
X_train_overall_mrna_imp["PATIENT_ID"] = train_all_overall_mrna["PATIENT_ID"].values
X_test_overall_mrna_imp["PATIENT_ID"] = test_all_overall_mrna["PATIENT_ID"].values
X_train_overall_mrna_imp = move_id_first(X_train_overall_mrna_imp)
X_test_overall_mrna_imp = move_id_first(X_test_overall_mrna_imp)

In [51]:
#integrate cohort gwas (N=1980,N=153)
Xc_train_overall_gwas = X_train_overall_gwas.iloc[:, :26]
Xg_train_overall_gwas = X_train_overall_gwas.iloc[:, 26:] #only impute on training
Xc_test_overall_gwas  = X_test_overall_gwas.iloc[:, :26]
Xg_test_overall_gwas  = X_test_overall_gwas.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_overall_gwas_scaled = scaler.fit_transform(Xg_train_overall_gwas) #std first

imputer = KNNImputer(n_neighbors=5,weights="distance",metric="nan_euclidean") #knn
Xg_train_overall_gwas_imp = imputer.fit_transform(Xg_train_overall_gwas_scaled)

Xg_train_overall_gwas_imp = pd.DataFrame( #get columns' name back
    Xg_train_overall_gwas_imp,
    columns=Xg_train_overall_gwas.columns,
    index=Xg_train_overall_gwas.index)

X_train_overall_gwas_imp = pd.concat([Xc_train_overall_gwas, Xg_train_overall_gwas_imp], axis=1)

Xg_test_overall_gwas_scaled = scaler.transform(Xg_test_overall_gwas) #Scale and std test using TRAIN statistics
Xg_test_overall_gwas_imp = imputer.transform(Xg_test_overall_gwas_scaled)
Xg_test_overall_gwas_imp = pd.DataFrame(
    Xg_test_overall_gwas_imp,
    columns=Xg_test_overall_gwas.columns,
    index=Xg_test_overall_gwas.index
)
X_test_overall_gwas_imp = pd.concat([Xc_test_overall_gwas, Xg_test_overall_gwas_imp], axis=1)

##add ID back
X_train_overall_gwas_imp["PATIENT_ID"] = train_all_overall_gwas["PATIENT_ID"].values
X_test_overall_gwas_imp["PATIENT_ID"]  = test_all_overall_gwas["PATIENT_ID"].values
def move_id_first(df, id_col="PATIENT_ID"): #move ID to the first column
    cols = [id_col] + [c for c in df.columns if c != id_col]
    return df[cols]
X_train_overall_gwas_imp = move_id_first(X_train_overall_gwas_imp)
X_test_overall_gwas_imp  = move_id_first(X_test_overall_gwas_imp)

In [31]:
#integrate cohort case 1(mrna only)
Xc_train_integrate_1_mrna = X_train_integrate_1_mrna.iloc[:, :26]
Xg_train_integrate_1_mrna = X_train_integrate_1_mrna.iloc[:, 26:] #only impute on training
Xc_test_integrate_1_mrna  = X_test_integrate_1_mrna.iloc[:, :26]
Xg_test_integrate_1_mrna  = X_test_integrate_1_mrna.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_integrate_1_mrna_scaled = scaler.fit_transform(Xg_train_integrate_1_mrna) #std first

imputer = KNNImputer(n_neighbors=5,weights="distance",metric="nan_euclidean") #knn
Xg_train_integrate_1_mrna_imp = imputer.fit_transform(Xg_train_integrate_1_mrna_scaled)

Xg_train_integrate_1_mrna_imp = pd.DataFrame( #get columns' name back
    Xg_train_integrate_1_mrna_imp,
    columns=Xg_train_integrate_1_mrna.columns,
    index=Xg_train_integrate_1_mrna.index)

X_train_integrate_1_mrna_imp = pd.concat([Xc_train_integrate_1_mrna, Xg_train_integrate_1_mrna_imp], axis=1)

Xg_test_integrate_1_mrna_scaled = scaler.transform(Xg_test_integrate_1_mrna) #Scale and std test using TRAIN statistics
Xg_test_integrate_1_mrna_imp = imputer.transform(Xg_test_integrate_1_mrna_scaled)
Xg_test_integrate_1_mrna_imp = pd.DataFrame(
    Xg_test_integrate_1_mrna_imp,
    columns=Xg_test_integrate_1_mrna.columns,
    index=Xg_test_integrate_1_mrna.index
)
X_test_integrate_1_mrna_imp = pd.concat([Xc_test_integrate_1_mrna, Xg_test_integrate_1_mrna_imp], axis=1)

##add ID back
X_train_integrate_1_mrna_imp["PATIENT_ID"] = train_all_integrate_1_mrna["PATIENT_ID"].values
X_test_integrate_1_mrna_imp["PATIENT_ID"]  = test_all_integrate_1_mrna["PATIENT_ID"].values
def move_id_first(df, id_col="PATIENT_ID"): #move ID to the first column
    cols = [id_col] + [c for c in df.columns if c != id_col]
    return df[cols]
X_train_integrate_1_mrna_imp = move_id_first(X_train_integrate_1_mrna_imp)
X_test_integrate_1_mrna_imp  = move_id_first(X_test_integrate_1_mrna_imp)

In [30]:
#integrate cohort case 2(mrna only)
Xc_train_integrate_2_mrna = X_train_integrate_2_mrna.iloc[:, :26]
Xg_train_integrate_2_mrna = X_train_integrate_2_mrna.iloc[:, 26:] #only impute on training
Xc_test_integrate_2_mrna  = X_test_integrate_2_mrna.iloc[:, :26]
Xg_test_integrate_2_mrna  = X_test_integrate_2_mrna.iloc[:, 26:]

scaler = StandardScaler()
Xg_train_integrate_2_mrna_scaled = scaler.fit_transform(Xg_train_integrate_2_mrna) #std first

imputer = KNNImputer(n_neighbors=5,weights="distance",metric="nan_euclidean") #knn
Xg_train_integrate_2_mrna_imp = imputer.fit_transform(Xg_train_integrate_2_mrna_scaled)

Xg_train_integrate_2_mrna_imp = pd.DataFrame( #get columns' name back
    Xg_train_integrate_2_mrna_imp,
    columns=Xg_train_integrate_2_mrna.columns,
    index=Xg_train_integrate_2_mrna.index)

X_train_integrate_2_mrna_imp = pd.concat([Xc_train_integrate_2_mrna, Xg_train_integrate_2_mrna_imp], axis=1)

Xg_test_integrate_2_mrna_scaled = scaler.transform(Xg_test_integrate_2_mrna) #Scale and std test using TRAIN statistics
Xg_test_integrate_2_mrna_imp = imputer.transform(Xg_test_integrate_2_mrna_scaled)
Xg_test_integrate_2_mrna_imp = pd.DataFrame(
    Xg_test_integrate_2_mrna_imp,
    columns=Xg_test_integrate_2_mrna.columns,
    index=Xg_test_integrate_2_mrna.index
)
X_test_integrate_2_mrna_imp = pd.concat([Xc_test_integrate_2_mrna, Xg_test_integrate_2_mrna_imp], axis=1)

##add ID back
X_train_integrate_2_mrna_imp["PATIENT_ID"] = train_all_integrate_2_mrna["PATIENT_ID"].values
X_test_integrate_2_mrna_imp["PATIENT_ID"]  = test_all_integrate_2_mrna["PATIENT_ID"].values
def move_id_first(df, id_col="PATIENT_ID"): #move ID to the first column
    cols = [id_col] + [c for c in df.columns if c != id_col]
    return df[cols]
X_train_integrate_2_mrna_imp = move_id_first(X_train_integrate_2_mrna_imp)
X_test_integrate_2_mrna_imp  = move_id_first(X_test_integrate_2_mrna_imp)

In [36]:
##output all dataset ready for prediction models
X_train_integrate_1_imp.to_csv("/Users/mac1/Desktop/X_train_integrate_1_imp.csv", index=False)
X_test_integrate_1_imp.to_csv("/Users/mac1/Desktop/X_test_integrate_1_imp.csv", index=False)
y_train_integrate_1.to_csv("/Users/mac1/Desktop/y_train_integrate_1.csv", index=False)
y_test_integrate_1.to_csv("/Users/mac1/Desktop/y_test_integrate_1.csv", index=False)

X_train_integrate_2_imp.to_csv("/Users/mac1/Desktop/X_train_integrate_2_imp.csv", index=False)
X_test_integrate_2_imp.to_csv("/Users/mac1/Desktop/X_test_integrate_2_imp.csv", index=False)
y_train_integrate_2.to_csv("/Users/mac1/Desktop/y_train_integrate_2.csv", index=False)
y_test_integrate_2.to_csv("/Users/mac1/Desktop/y_test_integrate_2.csv", index=False)

X_train_overall_imp.to_csv("/Users/mac1/Desktop/X_train_overall_imp.csv", index=False)
X_test_overall_imp.to_csv("/Users/mac1/Desktop/X_test_overall_imp.csv", index=False)
y_train_overall.to_csv("/Users/mac1/Desktop/y_train_overall.csv", index=False)
y_test_overall.to_csv("/Users/mac1/Desktop/y_test_overall.csv", index=False)

In [45]:
X_train_integrate_gwas_imp.to_csv("/Users/mac1/Desktop/X_train_integrate_gwas_imp.csv", index=False)
X_test_integrate_gwas_imp.to_csv("/Users/mac1/Desktop/X_test_integrate_gwas_imp.csv", index=False)
X_train_overall_gwas_imp.to_csv("/Users/mac1/Desktop/X_train_overall_gwas_imp.csv", index=False)
X_test_overall_gwas_imp.to_csv("/Users/mac1/Desktop/X_test_overall_gwas_imp.csv", index=False)
y_train_integrate_gwas.to_csv("/Users/mac1/Desktop/y_train_integrate_gwas.csv", index=False)
y_test_integrate_gwas.to_csv("/Users/mac1/Desktop/y_test_integrate_gwas.csv", index=False)
y_train_overall_gwas.to_csv("/Users/mac1/Desktop/y_train_overall_gwas.csv", index=False)
y_test_overall_gwas.to_csv("/Users/mac1/Desktop/y_test_overall_gwas.csv", index=False)

In [64]:
X_train_integrate_1cli_imp.to_csv("/Users/mac1/Desktop/X_train_integrate_1cli_imp.csv", index=False)
X_test_integrate_1cli_imp.to_csv("/Users/mac1/Desktop/X_test_integrate_1cli_imp.csv", index=False)
y_train_integrate_1cli.to_csv("/Users/mac1/Desktop/y_train_integrate_1cli.csv", index=False)
y_test_integrate_1cli.to_csv("/Users/mac1/Desktop/y_test_integrate_1cli.csv", index=False)

X_train_integrate_2cli_imp.to_csv("/Users/mac1/Desktop/X_train_integrate_2cli_imp.csv", index=False)
X_test_integrate_2cli_imp.to_csv("/Users/mac1/Desktop/X_test_integrate_2cli_imp.csv", index=False)
y_train_integrate_2cli.to_csv("/Users/mac1/Desktop/y_train_integrate_2cli.csv", index=False)
y_test_integrate_2cli.to_csv("/Users/mac1/Desktop/y_test_integrate_2cli.csv", index=False)

In [79]:
df=X_train_integrate_1cli_imp
n_methy = df.columns.str.endswith("_methy").sum()
n_mrna  = df.columns.str.endswith("_mrna").sum()
print("methy:", n_methy)
print("mrna :", n_mrna)

methy: 66
mrna : 131


In [71]:
X_train_overall_mrna_imp.to_csv("/Users/mac1/Desktop/X_train_overall_mrna_imp.csv", index=False)
X_test_overall_mrna_imp.to_csv("/Users/mac1/Desktop/X_test_overall_mrna_imp.csv", index=False)
y_train_overall_mrna.to_csv("/Users/mac1/Desktop/y_train_overall_mrna.csv", index=False)
y_test_overall_mrna.to_csv("/Users/mac1/Desktop/y_test_overall_mrna.csv", index=False)

In [32]:
X_train_integrate_1_mrna_imp.to_csv("/Users/mac1/Desktop/X_train_integrate_1_mrna_imp.csv", index=False)
X_test_integrate_1_mrna_imp.to_csv("/Users/mac1/Desktop/X_test_integrate_1_mrna_imp.csv", index=False)
y_train_integrate_1_mrna.to_csv("/Users/mac1/Desktop/y_train_integrate_1_mrna.csv", index=False)
y_test_integrate_1_mrna.to_csv("/Users/mac1/Desktop/y_test_integrate_1_mrna.csv", index=False)

X_train_integrate_2_mrna_imp.to_csv("/Users/mac1/Desktop/X_train_integrate_2_mrna_imp.csv", index=False)
X_test_integrate_2_mrna_imp.to_csv("/Users/mac1/Desktop/X_test_integrate_2_mrna_imp.csv", index=False)
y_train_integrate_2_mrna.to_csv("/Users/mac1/Desktop/y_train_integrate_2_mrna.csv", index=False)
y_test_integrate_2_mrna.to_csv("/Users/mac1/Desktop/y_test_integrate_2_mrna.csv", index=False)